In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import xarray as xr
import glob
import os

import py7zr

### Import shared files

In [2]:
covariate_metadata = pd.read_csv('../DATA/Stream_Temp/covariate_metadata.csv', nrows = 80)
covariate_metadata.head(5)

,COVARIATE,UNITS,SYMBOL,DESCRIPTION,DATA SOURCE (Retrospective),DATA SOURCE (GCM Scenarios)
0,COMID,unitless,NaN,unique identifier for matching to stream reach...,NHDv2 attributes; McKay et al. (2012),NaN
1,cov.antec_air_temp,C,Tl,Antecedent air temperature (summarized by the ...,Calculated; Siegel et al. (2023),Calculated as in Siegel et al. (2023)
2,cov.air_temp_mean_ws,C,Tws,mean daily air temperature in upstream watershed,PRISM; Di Luzio et al. (2008),RMJOC(2018); IPCC (2014); USCRT
3,cov.SWE_ws,mm,Sws,Mean daily snowpack snow-water equivalent (SWE...,National Snow and Ice Data Center; Broxton et ...,RMJOC(2018) intermediate modeling product
4,cov.daylength_hours,h,DL,daylength,DayMet; https://daymet.ornl.gov/,DayMet; https://daymet.ornl.gov/


In [3]:
gcm_metadata = pd.read_csv('../DATA/Stream_Temp/gcm_metadata.csv', nrows = 10)
gcm_metadata.head(5)

,Global Climate Model short name,GCM long name,Country,Agency,Coupled Model Intercomparison Project version,Emissions scenario,Downscaling method,Hydrology model,Network routing
0,CanESM2,Canadian Earth System Model version 2,Canada,Canadian Centre for Climate Modeling and Analysis,5,RCP 8.5,Multivariate Adaptive Constructed Analogs (Aba...,"Variable Infiltration Capacity, using paramete...",mizuRoute (Mizukami 2019)
1,CCSM4,Community Climate System Model version 4,USA,"National Center of Atmospheric Research, USA",5,RCP 8.5,Multivariate Adaptive Constructed Analogs (Aba...,"Variable Infiltration Capacity, using paramete...",mizuRoute (Mizukami 2019)
2,CNRM-CM5,National Center for Meteorological Research Mo...,France,"National Centre of Meteorological Research, Fr...",5,RCP 8.5,Multivariate Adaptive Constructed Analogs (Aba...,"Variable Infiltration Capacity, using paramete...",mizuRoute (Mizukami 2019)
3,CSIRO-Mk3-6-0,Commonwealth Scientific and Industrial Researc...,Australia,Commonwealth Scientific and Industrial Researc...,5,RCP 8.5,Multivariate Adaptive Constructed Analogs (Aba...,"Variable Infiltration Capacity, using paramete...",mizuRoute (Mizukami 2019)
4,GDFL-ESM2M,Global Fluid Dynamics Lab's Earth Systems Mode...,USA,"NOAA Geophysical Fluid Dynamics Laboratory, USA",5,RCP 8.5,Multivariate Adaptive Constructed Analogs (Aba...,"Variable Infiltration Capacity, using paramete...",mizuRoute (Mizukami 2019)


In [4]:
gcms = list(gcm_metadata.iloc[:, 0].values)

shortname_dict = {i+1:gcms[i] for i in list(range(10))}
shortname_dict

{1: 'CanESM2',
 2: 'CCSM4',
 3: 'CNRM-CM5',
 4: 'CSIRO-Mk3-6-0',
 5: 'GDFL-ESM2M',
 6: 'HadGEM-CC365',
 7: 'HadGEM2-ES',
 8: 'inmcm4',
 9: 'IPSL-CM5A-MR',
 10: 'MIROC5'}

In [6]:
# the following use ~ mm/21/yy dates for cutting (i.e. spring starts on March 21) - per Aimee

def assign_season(doy):
    """Assigns a season based on the day of year (doy)."""
    if 1 <= doy <= 79:
        return "winter"
    elif 80 <= doy <= 171:
        return "spring"
    elif 172 <= doy <= 263:
        return "summer"
    elif 264 <= doy <= 356:
        return "fall"
    else:
        return "winter"

In [7]:
# (~4 sec/HUC)
def getSeasonalAnomalies(indf, baseline_period: range, anomaly_periods: list, cols: list):
    cur_huc_anoms = []
    cur_comids = list(indf.COMID.unique())
    for j in cur_comids:
        cur_df = indf[indf.COMID == j][cols]
        cur_ens_med = cur_df.groupby('date').median()
        cur_ens_med['doy'] = cur_ens_med.index.dayofyear
        cur_ens_med['season'] = cur_ens_med['doy'].apply(assign_season)

        cur_anomalies = {}
        baseline = cur_ens_med[cur_ens_med.index.year.isin(baseline_period)].groupby('season').median()
        cur_anomalies[f'{baseline_period[0]}s'] = baseline
        
        for per in anomaly_periods:
            cur_anomalies[f'{per[0]}s'] = cur_ens_med[cur_ens_med.index.year.isin(per)].groupby('season').median().subtract(baseline)
        cur_seas_anomalies = pd.concat(cur_anomalies, axis = 1)
        cur_huc_anoms.append(cur_seas_anomalies)
    return pd.concat(cur_huc_anoms, keys = cur_comids)

In [8]:
temp_dir = '/Volumes/Elements/URycki/temp_data/'

In [9]:
periods = {'2010s': list(range(2010, 2020)),
           '2020s': list(range(2020, 2030)), 
           '1990s': list(range(1990, 2000)), 
           '2050s': list(range(2050, 2060)), 
           '2080s': list(range(2080, 2090))}

In [10]:
temp_dir_gcms = os.path.join(temp_dir, 'preds_GCM/')
temp_files_gcms = glob.glob(temp_dir_gcms + '*.nc')
print(len(temp_files_gcms))
temp_files_gcms.sort()

1316


In [11]:
hucs = [f[-13:-3] for f in temp_files_gcms]

### Temperature anomalies

In [20]:
# Temp anomalies
for cur_huc in hucs[:2]:
    print(cur_huc)

    preds = xr.open_dataset(f'/Volumes/Elements/URycki/temp_data/preds_GCM/{cur_huc}.nc', decode_times=True)
    predsdf = preds.to_dataframe(dim_order = ['COMID', 'GCM', 'date'])
    preds_flat = predsdf.reset_index()

    huc_anoms = getSeasonalAnomalies(preds_flat, baseline_period = periods['2010s'], 
                            anomaly_periods = [periods[key] for key in ('1990s', '2050s', '2080s')], cols = ['COMID', 'date', 'prd.stream_temp'])
    huc_anoms_reindex = huc_anoms.drop(['doy', 'COMID'], level = 1, axis = 1)    
    
    huc_anoms_reindex.reset_index(names = ['COMID', 'season']).to_csv(
        os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.csv'), index = False) #, compression = 'zip')

1701010107
1701010108


In [23]:
pd.read_csv(os.path.join(temp_dir_gcms, 'seasonal_anomolies', f'{cur_huc}_anoms.csv'), header=[0,1])

,COMID,season,2010s,1990s,2050s,2080s
,Unnamed: 0_level_1,Unnamed: 1_level_1,prd.stream_temp,prd.stream_temp,prd.stream_temp,prd.stream_temp
0,22878623,fall,3.921150,-0.299178,1.096812,2.240112
1,22878623,spring,8.469155,-0.411885,1.633280,2.942516
2,22878623,summer,15.968426,-0.946710,1.242376,2.557516
3,22878623,winter,0.794843,-0.155174,0.526287,1.571385
4,22878625,fall,3.965583,-0.296498,1.080081,2.268231
...,...,...,...,...,...,...
123,22879895,winter,0.323003,0.103842,0.419040,1.216069
124,22879989,fall,2.040540,-0.095128,0.127186,0.459159
125,22879989,spring,3.544079,-0.016492,0.421183,0.953634


In [24]:
# Select columns with 'float64' dtype  
float64_cols = list(huc_anoms.select_dtypes(include='float64'))

# The same code again calling the columns
huc_anoms[float64_cols] = huc_anoms[float64_cols].astype('float32')

In [25]:
huc_anoms.dtypes

2010s  COMID              float32
       prd.stream_temp    float32
       doy                float32
1990s  COMID              float32
       prd.stream_temp    float32
       doy                float32
2050s  COMID              float32
       prd.stream_temp    float32
       doy                float32
2080s  COMID              float32
       prd.stream_temp    float32
       doy                float32
dtype: object

### Covariate anomalies

In [12]:
%load_ext rpy2.ipython

In [13]:
%%R
install.packages("fst")

R[write to console]: Installing package into ‘/Users/dawn.urycki/Library/R/x86_64/4.3/library’
(as ‘lib’ is unspecified)



--- Please select a CRAN mirror for use in this session ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Austria (Wien 1) [https]
 6: Belgium (Brussels) [https]
 7: Brazil (PR) [https]
 8: Brazil (SP 1) [https]
 9: Brazil (SP 2) [https]
10: Bulgaria [https]
11: Canada (MB) [https]
12: Canada (ON 1) [https]
13: Canada (ON 2) [https]
14: Chile (Santiago) [https]
15: China (Beijing 2) [https]
16: China (Beijing 3) [https]
17: China (Hefei) [https]
18: China (Hong Kong) [https]
19: China (Jinan) [https]
20: China (Lanzhou) [https]
21: China (Nanjing) [https]
22: China (Shanghai 2) [https]
23: China (Shenzhen) [https]
24: China (Wuhan) [https]
25: Colombia (Cali) [https]
26: Costa Rica [https]
27: Cyprus [https]
28: Czech Republic [https]
29: Denmark [https]
30: East Asia [https]
31: Ecuador (Cuenca) [https]
32: France (Lyon 1) [https]
33: France (Lyon 2) [https]
34: France (Marseille) 

Selection:  70


R[write to console]: trying URL 'https://mirrors.nics.utk.edu/cran/src/contrib/fst_0.9.8.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 274495 bytes (268 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to co

x86_64-apple-darwin13.4.0-clang++ -std=gnu++17 -I"/opt/miniconda3/envs/temp-data-env/lib/R/include" -DNDEBUG  -I'/Users/dawn.urycki/Library/R/x86_64/4.3/library/Rcpp/include' -I'/Users/dawn.urycki/Library/R/x86_64/4.3/library/fstcore/include' -D_FORTIFY_SOURCE=2 -isystem /opt/miniconda3/envs/temp-data-env/include -mmacosx-version-min=10.13 -mmacosx-version-min=10.13 -I/opt/miniconda3/envs/temp-data-env/include    -fPIC  -march=core2 -mtune=haswell -mssse3 -ftree-vectorize -fPIC -fstack-protector-strong -O2 -pipe -stdlib=libc++ -fvisibility-inlines-hidden -fmessage-length=0 -isystem /opt/miniconda3/envs/temp-data-env/include -fdebug-prefix-map=/Users/runner/miniforge3/conda-bld/r-base-split_1728291276454/work=/usr/local/src/conda/r-base-4.3.3 -fdebug-prefix-map=/opt/miniconda3/envs/temp-data-env=/usr/local/src/conda-prefix  -c RcppExports.cpp -o RcppExports.o
x86_64-apple-darwin13.4.0-clang++ -std=gnu++17 -I"/opt/miniconda3/envs/temp-data-env/lib/R/include" -DNDEBUG  -I'/Users/dawn.uryc

installing to /Users/dawn.urycki/Library/R/x86_64/4.3/library/00LOCK-fst/00new/fst/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (fst)
R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/private/var/folders/95/99dy_ns968195cnlvp5xhwqm0000gp/T/RtmpbfF35G/downloaded_packages’
R[write to console]: 
R[write to console]: 



In [14]:
%%R
library(fst)
getwd()

R[write to console]: fst package v0.9.8



[1] "/Users/dawn.urycki/Repos/temp-data"


In [15]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [16]:
colsToUse = ['COMID', 'date', 'cov.air_temp_ws', 'cov.Flow_log', 'cov.SWE_ws']

In [ ]:
%%time
fail_list = {}

for i in range(len(hucs))[68:]:
    cur_huc = hucs[i]
    print(cur_huc)

    gcm_frames = []
    for cur_gcm in gcms:
        try: gcm_frames.append(ro.r(f"read.fst('/Volumes/Elements/URycki/temp_data/covariates_GCM/{cur_gcm}/huc_{cur_huc}.fst')"))
        except: fail_list['cur_huc'] = cur_gcm  # update this to check if fail_list['cur_huc'] exists; append cur_gcm
        
    if (cur_huc not in fail_list.keys()) & (gcm_frames != []):
        df = pd.concat(gcm_frames)
        df['date'] = pd.to_datetime(df['tim.year'] * 1000 + df['tim.doy'], format='%Y%j')
        print(df.shape)  

        huc_cov_anoms = getSeasonalAnomalies(df, baseline_period = periods['2010s'], 
                            anomaly_periods = [periods[key] for key in ('1990s', '2050s', '2080s')], cols = colsToUse)
        
        huc_cov_anoms_reindex = huc_cov_anoms.drop(['doy', 'COMID'], level = 1, axis = 1)

        huc_cov_anoms_reindex.reset_index(names = ['COMID', 'season']).to_csv(
            os.path.join(temp_dir_gcms, 'seasonal_covariate_anomalies', f'{cur_huc}_cov_anoms.zip'), index = False, compression = 'zip')

R[write to console]: fstcore package v0.9.18

R[write to console]: (OpenMP detected, using 16 threads)



1701020406
(41638120, 32)
1701020407
(25749890, 32)
1701020408
(45473210, 32)
1701020501
(16436100, 32)
1701020502
(5478700, 32)
1701020503
(29037110, 32)
1701020504
(14792490, 32)
1701020505
(25749890, 32)
1701020507
(12601010, 32)
1701020508
(48212560, 32)
1701020509
(16436100, 32)
1701020510
(13148880, 32)
1701020511
(18079710, 32)
1701020512
(2191480, 32)
1701020514
(30680720, 32)
1701020515
(65196530, 32)
1701020516
(18079710, 32)
1701020601
(1095740, 32)
1701020603
(14792490, 32)
1701020604
(41638120, 32)
1701020605
(41638120, 32)
1701020606
(47116820, 32)
1701020701
(21914800, 32)
1701020702
(25202020, 32)
1701020703
(39994510, 32)
1701020704
(34515810, 32)
1701020705
(36707290, 32)
1701020801
(29584980, 32)
1701020802
(55882740, 32)
1701020803
(6026570, 32)
1701020804
(3287220, 32)
1701020805
(19175450, 32)
1701020901
(37255160, 32)
1701020902
(14244620, 32)
1701020903
(33967940, 32)
1701020904
(30132850, 32)
1701020905
(17531840, 32)
1701020906
(18079710, 32)
1701020907
(26297

In [21]:
i = 68

cur_huc = hucs[i]
cur_huc

'1701020406'

In [109]:
pd.read_csv(os.path.join(temp_dir_gcms, 'seasonal_covariate_anomalies', f'{cur_huc}_cov_anoms.zip'), compression="zip", header = [0,1])

COMID             season           2010s               \
    Unnamed: 0_level_1 Unnamed: 1_level_1 cov.air_temp_ws cov.Flow_log   
0             22878623               fall        0.154336     1.350489   
1             22878623             spring        7.303487     2.974164   
2             22878623             summer       15.877007     2.403840   
3             22878623             winter       -3.494835     1.140771   
4             22879591               fall        0.064926     1.247175   
..                 ...                ...             ...          ...   
123           22878839             winter       -6.220603    -2.301261   
124           22878841               fall       -2.465904    -1.454547   
125           22878841             spring        4.622115    -0.775237   
126           22878841             summer       13.242865    -0.467143   
127           22878841             winter       -6.220603    -2.301261   

                          1990s                                   2050s  \
     cov.SWE_ws cov.air_temp_ws cov.Flow_log cov.SWE_ws cov.air_temp_ws   
0      5.497962       -0.540583     0.184470   3.612159        1.827772   
1    126.146501       -0.597353    -0.015124  13.194312        2.160125   
2      0.000000       -1.443549     0.121384   0.000000        2.408694   
3    177.201229       -0.737976     0.074986  16.034191        1.331160   
4      5.228977       -0.529118     0.189095   3.633608        1.817388   
..          ...             ...          ...        ...             ...   
123  558.792386       -0.768746     0.121025   9.262350        1.471665   
124   26.842200       -0.655865     0.133457  15.063381        1.689044   
125  588.546166       -0.608738    -0.153506  13.483736        2.103617   
126    0.000000       -1.592515     0.097331   0.000000        2.330793   
127  558.792386       -0.768746     0.121025   9.262350        1.471665   

                                       2080s                           
    cov.Flow_log  cov.SWE_ws cov.air_temp_ws cov.Flow_log  cov.SWE_ws  
0      -0.164333   -3.193326        3.617297    -0.168122   -5.048572  
1       0.095225  -50.116231        3.617544     0.033491  -89.552106  
2      -0.325561    0.000000        5.329137    -0.550801    0.000000  
3       0.264473  -28.815278        3.580706     0.870299  -71.454419  
4      -0.164334   -3.061319        3.625897    -0.164469   -4.811189  
..           ...         ...             ...          ...         ...  
123    -0.087628   -8.048471        3.625162     0.215578  -95.455069  
124    -0.186862  -14.797624        3.506235    -0.270281  -23.478210  
125     0.589394 -197.353268        3.592211     0.576021 -373.603200  
126    -0.240239    0.000000        5.133570    -0.475333    0.000000  
127    -0.087628   -8.048471        3.625162     0.215578  -95.455069  

[128 rows x 14 columns]

In [391]:
undef_list = list(set(cur_preds.columns).difference(set(covariate_metadata.COVARIATE)))
undef = [str(i) for i in undef_list]
undef.sort()
undef

['GCM',
 'SegID',
 'cov.Flow',
 'cov.Flow_log',
 'cov.SWE',
 'cov.SWE_last_doy',
 'cov.SWE_mean_year',
 'cov.air_temp',
 'cov.air_temp_12d',
 'cov.air_temp_15d',
 'cov.air_temp_18d',
 'cov.air_temp_21d',
 'cov.air_temp_24d',
 'cov.air_temp_27d',
 'cov.air_temp_30d',
 'cov.air_temp_35d',
 'cov.air_temp_3d',
 'cov.air_temp_40d',
 'cov.air_temp_50d',
 'cov.air_temp_60d',
 'cov.air_temp_6d',
 'cov.air_temp_9d',
 'cov.air_temp_ws',
 'cov.daylength',
 'date',
 'lookup',
 'tim.date',
 'tim.year']

In [400]:
cur_preds.columns

Index(['GCM', 'COMID', 'SegID', 'lookup', 'cov.air_temp', 'cov.air_temp_ws',
       'cov.air_temp_3d', 'cov.air_temp_6d', 'cov.air_temp_9d',
       'cov.air_temp_12d', 'cov.air_temp_15d', 'cov.air_temp_18d',
       'cov.air_temp_21d', 'cov.air_temp_24d', 'cov.air_temp_27d',
       'cov.air_temp_30d', 'cov.air_temp_35d', 'cov.air_temp_40d',
       'cov.air_temp_50d', 'cov.air_temp_60d', 'cov.SWE', 'cov.SWE_ws',
       'cov.Flow', 'cov.Flow_log', 'tim.year', 'tim.doy', 'tim.date',
       'cov.daylength', 'cov.daylength_hours', 'cov.SWE_mean_year',
       'cov.SWE_1Apr', 'cov.SWE_last_doy', 'date'],
      dtype='object')